In [2]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime 
import os
import time

In [3]:
fecha_actual = datetime.now().strftime('%Y-%m-%d')

## Web Scrapping IMSSS

### Web Scrapping de la página general por años

In [12]:
# Guardamos el url base para poder concatenar los links de los archivos
base_url = "http://datos.imss.gob.mx"
# Obtenemos el contenido de la página
r = requests.get("http://datos.imss.gob.mx/dataset")
# Creamos el objeto soup
soup = BeautifulSoup(r.content, "html.parser")
# Obtenemos todos los links de la página
spans = soup.find_all("a", {'data-format':'csv'}, href=True)
# Se guardaran los links de los archivos que nos interesan por año
links = []
# Obtenemos los links de los archivos que nos interesan
for link in spans:
    href = link.get("href")
    if '/dataset/asg-' in href:
        condicion = int(re.findall(r'\d{4}', href)[0])
        if condicion >= 2006 and condicion < 2023:
            links.append(base_url + href)

#### Descripción de fuentes

In [13]:
# Crea y abre el archivo donde se excribe la información de la referncia
ruta_archivo = os.path.join('..', 'descripciones', 'imss_descripcion.txt')
with open(ruta_archivo, 'w+', encoding='utf-8') as file:
    file.write('Datos del IMSS\n')
    for link in links:
        time.sleep(5)
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        file.write('Enlace de archivos por mes: '+link+'\n')
        # Tabla de referencias
        table = soup.find('table', class_= 'field-group-format group_additional')
        # Moviendose por filas
        rows = table.find_all('tr')
        for row in rows[1:]:
            if row:
                cells = row.find_all(['th', 'td'])
                # Extrae el texto de las celdas y los concatena
                file.write(': '.join(cell.get_text(strip=True) for cell in cells))
                file.write('\n')
    # Agrega la fecha en la que se corrió el script que extrajo los datos
    file.write('\nFecha de descarga: '+fecha_actual+'\n')
    file.close()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


### Web Scrapping parte 2 por meses

In [32]:
# Por cada año, guardaremos los links de los archivos que nos interesan por meses
links_2 = []
# Obtenemos los links de los archivos que nos interesan
for i in range(len(links)):
    time.sleep(5)
    r_2 = requests.get(links[i])
    soup_2 = BeautifulSoup(r_2.content, "html.parser")
    spans_2 = soup_2.find_all("a", {'property':'dcat:accessURL'}, href=True)
    for link in spans_2:
        links_2.append(base_url + link.get("href"))

In [38]:
# Creamos una lista para guardar los dataframes
df = []
# Obtenemos los links de los archivos que nos interesan
for i in range(len(links_2)):
    time.sleep(5)
    r_3 = requests.get(links_2[i])
    soup_3 = BeautifulSoup(r_3.content, "html.parser")
    spans_3 = soup_3.find_all("a", href=True)
    for link in spans_3:
        if '.csv' in link.get("href"):
            df.append(link.get("href"))

In [40]:
# Crear un DataFrame vacío para almacenar los datos combinados
df_combined = pd.DataFrame()

for i in range(len(df)):
    # Leer el archivo CSV y especificar el tipo de datos adecuado (si es necesario)
    df_csv = pd.read_csv(df[i], encoding="latin-1", sep="|", dtype={"columna_con_tipo_mezclado": str}, low_memory=False)

    columns = ['rango_uma', 'ta', 'teu', 'tec', 'tpu', 'tpc',
               'ta_sal', 'teu_sal', 'tec_sal', 'tpu_sal', 'tpc_sal', 'masa_sal_ta',
               'masa_sal_teu', 'masa_sal_tec', 'masa_sal_tpu', 'masa_sal_tpc', "sector_economico_1"]

    # Usar copy() para evitar SettingWithCopyWarning
    df_sonora = df_csv.loc[(df_csv["sector_economico_1"] == 0) & (df_csv["cve_entidad"] == 26)].copy()

    # Eliminar columnas
    df_sonora.drop(columns, inplace=True, axis=1, errors='ignore')

    # Extraer año y mes de la URL
    df_sonora['año'] = re.findall(r'\d{4}', df[i])[0]
    df_sonora['mes'] = re.findall(r'-(\d{2})-', df[i])[0]

    # Filtrar por sector_economico_2
    df_append = df_sonora[df_sonora["sector_economico_2"].isin([1, 2, 4])]

    # Usar pd.concat en lugar de append (sin ignore_index=True)
    df_combined = pd.concat([df_combined, df_append])

/tmp/ipykernel_34140/1681092507.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, df_append])


In [43]:
ruta_archivo = os.path.join('..', 'data')
if not os.path.exists(ruta_archivo):
    os.makedirs(ruta_archivo)
ruta = os.path.join('..', 'data','df_imss.csv')
df_combined.to_csv(ruta) 

### Diccionario y catálogo

In [19]:
dic_cat = pd.read_excel('http://datos.imss.gob.mx/sites/default/files/diccionario_de_datos_1.xlsx', sheet_name=None)

for sheet_name, sheet_data in dic_cat.items():
    if sheet_name == 'ejemplo archivo':
        continue  # Ignorar esta hoja y pasar a la siguiente
    elif sheet_name == 'Layout':
        # Guarda el layout en un archivo de Excel
        sheet_data = dic_cat['Layout']
        sheet_data.to_excel('diccionario_IMSS.xlsx')       
    else:
        # Reemplaza los espacios en el nombre de la hoja con guiones bajos
        nombre_archivo_csv = f"imss_{sheet_name.replace(' ', '_')}.csv"
        # Guarda el archivo CSV en el directorio
        ruta_archivo = os.path.join('..','data','imss_cat')
        if not os.path.exists(ruta_archivo):
            os.makedirs(ruta_archivo)
        ruta_archivo = os.path.join('..','data','imss_cat', nombre_archivo_csv)
        sheet_data.to_csv(ruta_archivo)
        print(f"La hoja '{sheet_name}' se ha guardado en '{nombre_archivo_csv}'")


La hoja 'delegación-subdelegación' se ha guardado en 'imss_delegación-subdelegación.csv'
La hoja 'entidad-municipio' se ha guardado en 'imss_entidad-municipio.csv'
La hoja 'sector 1' se ha guardado en 'imss_sector_1.csv'
La hoja 'sector 2' se ha guardado en 'imss_sector_2.csv'
La hoja 'sector 4' se ha guardado en 'imss_sector_4.csv'
La hoja 'Tamaño de registro patronal' se ha guardado en 'imss_Tamaño_de_registro_patronal.csv'
La hoja 'sexo' se ha guardado en 'imss_sexo.csv'
La hoja 'Rango edad' se ha guardado en 'imss_Rango_edad.csv'
La hoja 'Rango salario' se ha guardado en 'imss_Rango_salario.csv'
La hoja 'Rango UMA' se ha guardado en 'imss_Rango_UMA.csv'


## Función para catálogos

In [30]:
def catalogos(tipo_catalogo, soup):
    # Utilizar una expresión regular para buscar elementos <a> con 'title' que contiene 'Catalogos'
    pattern = re.compile(r'Cat[aá]logos', re.IGNORECASE)  # La opción re.IGNORECASE hace que la búsqueda sea insensible a mayúsculas/minúsculas
    enlace = soup.find_all("a", {'title':pattern}, href=True)
    for link in enlace:
        href = link.get("href")
        df = pd.read_excel(href, sheet_name=None)
        direc = os.path.join('..','data')
        if not os.path.exists(direc):
                os.makedirs(direc)
        # Se guarda el archivo
        nombre_archivo = os.path.join('..','data', f"cat_{tipo_catalogo}.xlsx")
        response = requests.get(href)
        with open(nombre_archivo, 'wb') as f:
                f.write(response.content)
        # Iteramos a través de las hojas del archivo
        for sheet_name, sheet_data in df.items():
            # Genera un nombre de archivo CSV basado en el nombre de la hoja
            nombre_archivo_csv = f"{tipo_catalogo}_{sheet_name}.csv"
            # Se guarda en
            ruta = os.path.join('..', 'data',f'{tipo_catalogo}')
            if not os.path.exists(ruta):
                os.makedirs(ruta)
            # Guarda el DataFrame de la hoja en un archivo CSV
            ruta_archivo = os.path.join(ruta, nombre_archivo_csv)
            sheet_data.to_csv(ruta_archivo, index = False)
            print(f"La hoja '{sheet_name}' se ha guardado en '{nombre_archivo_csv}'")

## Web Scraping Agricultura (2018-2022)

In [31]:
# Guardamos el url
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-de-agricultura-sonora/1573'
# Obtenemos el contenido de la página
r = requests.get(url)
# Creamos el objeto soup
soup = BeautifulSoup(r.content, "html.parser")
# Obtenemos todos los links de la página
spans = soup.find_all("a", href=True)

# Información de la fuente
ruta = os.path.join('..', 'descripciones','agricultura_descripcion.txt')
file = open(ruta, 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()

In [32]:
# Llamamos a la función catalogos
catalogos('Agricultura', soup)

La hoja 'ClasificaciónCultivos' se ha guardado en 'Agricultura_ClasificaciónCultivos.csv'
La hoja 'CiclosCultivos' se ha guardado en 'Agricultura_CiclosCultivos.csv'
La hoja 'Distritos' se ha guardado en 'Agricultura_Distritos.csv'
La hoja 'Municipios' se ha guardado en 'Agricultura_Municipios.csv'
La hoja 'Regiones' se ha guardado en 'Agricultura_Regiones.csv'


In [33]:
# Lista de años
years = [str(year) for year in range(2006,2021)]
links_xlsx = []
# Recorre la lista de años y busca enlaces con títulos que coincidan con cada año
for year in years:
    links = soup.find_all("a", {'title': f'Producción agrícola {year}'}, href=True)
    for link in links:
        links_xlsx.append(link['href'])

# Creamos un DataFrame vacío para almacenar los datos combinados
df_agricultura = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_agricultura = pd.concat([df_agricultura, pd.read_excel(links_xlsx[i])])

df_2021 = pd.read_excel('../data/extra/Agricultura Sonora 2021.xlsx')
df_2022 = pd.read_excel('../data/extra/Agricultura Sonora 2022.xlsx')

df_agricultura = pd.concat([df_agricultura, df_2021])
df_agricultura = pd.concat([df_agricultura, df_2022])

# Convertimos el df a csv
ruta = os.path.join('..','data','df_agricultura.csv')
df_agricultura.to_csv(ruta)


## Web Scraping Pecuaria (2018-2022)

In [34]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-ganaderia-sonora/1581'
# Realiza la solicitud HTTP
r = requests.get(url)
# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")

# Información de la fuente
ruta = os.path.join('..', 'descripciones','pecuaria_descripcion.txt')
file = open(ruta, 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()

In [35]:
# Llamamos a la función catalogos
catalogos('Ganaderia', soup)

La hoja 'Catálogo_Ganadería' se ha guardado en 'Ganaderia_Catálogo_Ganadería.csv'
La hoja 'Distritos' se ha guardado en 'Ganaderia_Distritos.csv'
La hoja 'Municipios' se ha guardado en 'Ganaderia_Municipios.csv'
La hoja 'Regiones' se ha guardado en 'Ganaderia_Regiones.csv'


In [36]:
# Lista de años
years = [str(year) for year in range(2006,2021)]
# Lista para almacenar los enlaces .xlsx
links_xlsx = []
# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')
# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

# Creamos un DataFrame vacío para almacenar los datos combinados
df_pecuaria = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pecuaria = pd.concat([df_pecuaria, pd.read_excel(links_xlsx[i])])

df_2021 = pd.read_excel('../data/extra/Cierre pecuario 2021.xlsx')
df_2022 = pd.read_excel('../data/extra/Cierre pecuario 2022.xlsx')

df_pecuaria = pd.concat([df_pecuaria, df_2021])
df_pecuaria = pd.concat([df_pecuaria, df_2022])

# Convertimos el df a csv
ruta = os.path.join('..','data','df_pecuaria.csv')
df_pecuaria.to_csv(ruta)

## Web Scraping Pesca (2018-2022)

In [37]:
# URL de la página HTML
url = 'https://datos.sonora.gob.mx/conjuntos-de-datos/mostrar/datos-pesca-sonora/1582'

# Realiza la solicitud HTTP
r = requests.get(url)

# Parsea el contenido HTML
soup = BeautifulSoup(r.content, "html.parser")
# Información de la fuente
ruta = os.path.join('..', 'descripciones','pesca_descripcion.txt')
file = open(ruta, 'w+', encoding = 'utf-8')
file.write('Enlace de los archivos: '+url+'\n')
data_info = soup.find('p', class_='mb-3').get_text()
info = data_info.split(' | ')
file.write('\n'.join(info))
file.write('\nFecha de descarga: '+fecha_actual)

file.close()


In [38]:
# Llamamos a la función catalogos
catalogos('Pesca', soup)

La hoja 'Especie_Pesca' se ha guardado en 'Pesca_Especie_Pesca.csv'
La hoja 'Distritos' se ha guardado en 'Pesca_Distritos.csv'
La hoja 'Municipios' se ha guardado en 'Pesca_Municipios.csv'
La hoja 'Regiones' se ha guardado en 'Pesca_Regiones.csv'


In [39]:
# Lista de años
years = [str(year) for year in range(2006,2021)]
# Lista para almacenar los enlaces .xlsx
links_xlsx = []
# Expresión regular para buscar enlaces .xlsx
xlsx_pattern = re.compile(r'\.xlsx$')
# Busca todos los enlaces con extensión .xlsx en la página
enlaces = soup.find_all('a', href=xlsx_pattern)

# Recorre los enlaces y verifica si el texto del enlace contiene un año de la lista
for enlace in enlaces:
    texto_enlace = enlace.text
    for year in years:
        if year in texto_enlace:
            links_xlsx.append(enlace['href'])

# Elimina duplicados (si los hay) utilizando set y luego convierte de nuevo a lista
links_xlsx = list(set(links_xlsx))

# Creamos un DataFrame vacío para almacenar los datos combinados
df_pesca = pd.DataFrame()
for i in range(len(links_xlsx)):
    df_pesca = pd.concat([df_pesca, pd.read_excel(links_xlsx[i])])

df_2021 = pd.read_excel('../data/extra/Datos pesca 2021.xlsx')
df_2022 = pd.read_excel('../data/extra/Datos pesca 2022.xlsx')

df_pesca = pd.concat([df_pesca, df_2021])
df_pesca = pd.concat([df_pesca, df_2022])


# Convertimos el df a csv
ruta = os.path.join('..','data','df_pesca.csv')
df_pesca.to_csv(ruta)